In [ ]:
!pip install langchain langchain-google-genai tavily-python brevo-python python-dotenv
!pip install langchain-community


In [11]:
#import libraries
import os
from dotenv import load_dotenv
import ast  # Abstract Syntax Tree library to safely evaluate strings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain.tools import tool
import brevo_python
from brevo_python.rest import ApiException
from langchain_community.tools.tavily_search import TavilySearchResults
from google.colab import userdata

# --- 1. SETUP: LOAD ENVIRONMENT VARIABLES ---
#set user key from colab secret manager.
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['BREVO_API_KEY'] = userdata.get('BREVO_API_KEY')

In [ ]:
# --- 2. FUNCTION DEFINITIONS: DEFINE THE WORKFLOW STEPS ---

def search_and_summarize_marketing_news():
    """
    Searches for the latest marketing news and returns a cohesive summary.
    """
    # Step 1: Search for the news
    print("--- Searching for news... ---")
    tavily_tool = TavilySearchResults(max_results=7)
    results = tavily_tool.invoke({"query": "latest marketing news and trends"})

    # Step 2: Summarize the news with an LLM
    print("--- Summarizing articles... ---")
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    article_text = "\n\n".join([f"Article URL: {article['url']}\nContent: {article['content']}" for article in results])
    prompt_template = f"""
    You are an expert marketing analyst. Summarize the key trends, takeaways, and announcements
    from the following marketing news articles. Structure the summary with clear headings for each topic.
    For each key point, cite the source URL.

    Here are the articles to summarize:
    {article_text}

    Final Summary:
    """
    summary = llm.invoke(prompt_template)
    return summary.content

def send_summary_email(summary: str, recipient_email: str):
    """Sends an email with the news summary using the Brevo API."""
    print(f"--- Preparing to send email to {recipient_email}... ---")
    configuration = brevo_python.Configuration()
    configuration.api_key['api-key'] = os.getenv("BREVO_API_KEY")
    api_instance = brevo_python.TransactionalEmailsApi(brevo_python.ApiClient(configuration))

    # IMPORTANT: Replace with your verified sender email from Brevo
    sender_email = "siangouweijoseph@gmail.com"
    subject = "Your Weekly Marketing News Digest (from your AI Workflow)"
    sender = {"name": "AI Marketing Assistant", "email": sender_email}
    to = [{"email": recipient_email}]
    send_smtp_email = brevo_python.SendSmtpEmail(to=to, sender=sender, subject=subject, text_content=summary)

    try:
        api_instance.send_transac_email(send_smtp_email)
        print("--- Email sent successfully! ---")
    except ApiException as e:
        print(f"--- Failed to send email. Error: {e} ---")


# --- 3. RUN THE WORKFLOW ---
if __name__ == "__main__":
    # IMPORTANT: Replace with your email address
    your_email_address = "siangouweijoseph@gmail.com"

    print("Starting the marketing news workflow...")
    # Step 1: Get the summary
    news_summary = search_and_summarize_marketing_news()

    # Step 2: Send the summary in an email
    send_summary_email(summary=news_summary, recipient_email=your_email_address)

    print("Workflow has finished its task.")


